# Лабораторна робота №6: ANFIS для варіанту №1

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


In [ ]:
df = pd.read_csv("anfis_variant1_data.csv")
df.head()


In [ ]:
X = df[['IOC(0)', 'IPC(0)', 'KVVE(-7)', 'M2(-7)']].values
y = df['IPC(+1)'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


### Подальші кроки:

Для реалізації ANFIS у Python можна скористатися бібліотекою `anfis`, наприклад `torch-anfis`. Вона потребує визначення:

- функцій належності (membership functions),
- нечітких правил,
- навчального циклу.

Рекомендую встановити бібліотеку `anfis` або `sugeGTD/anfis` (через pip), або ж імпортувати її локально, якщо вона вже у вас є.


## Навчання ANFIS-моделі з 10, 20 та 40 правилами

In [ ]:
# Встановлення бібліотеки anfis (якщо потрібно)
# pip install git+https://github.com/SantiagoHerre/anfis.git

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_squared_error

from anfis.model import AnfisNet
from anfis.membership import BellMembFunc, make_bell_mfs

# Підготовка тензорів
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

def train_anfis_model(n_rules):
    # Визначення функцій належності (Bell)
    input_mfs = []
    for i in range(X.shape[1]):
        input_mfs.append(
            [BellMembFunc(a=1, b=1, c=np.random.rand()*0.5 + 0.5) for _ in range(n_rules)]
        )

    model = AnfisNet(n_inputs=X.shape[1], n_rules=n_rules, input_mfs=input_mfs)
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    criterion = nn.MSELoss()

    n_epochs = 300
    loss_history = []

    for epoch in range(n_epochs):
        model.train()
        optimizer.zero_grad()
        y_pred = model(X_train_tensor)
        loss = criterion(y_pred, y_train_tensor)
        loss.backward()
        optimizer.step()
        loss_history.append(loss.item())

    return model, loss_history


In [ ]:
# Побудова моделей з різною кількістю правил
results = {}
for n_rules in [10, 20, 40]:
    model, loss_history = train_anfis_model(n_rules)
    with torch.no_grad():
        y_pred = model(X_test_tensor).numpy().flatten()
        mse = mean_squared_error(y_test, y_pred)
    results[n_rules] = {"model": model, "loss": loss_history, "mse": mse}
    print(f"Кількість правил: {n_rules}, MSE: {mse:.4f}")


In [ ]:
# Візуалізація
plt.figure(figsize=(10, 6))
for n_rules, res in results.items():
    plt.plot(res["loss"], label=f"{n_rules} правил")
plt.title("Зміна MSE loss під час навчання")
plt.xlabel("Епоха")
plt.ylabel("MSE Loss")
plt.legend()
plt.grid(True)
plt.show()
